In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

NameError: name 'libraries' is not defined

In [2]:
!pip install spleeter torch librosa tqdm pyworld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 6.7 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 94.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.

In [ ]:
import os
import librosa
import numpy as np
from spleeter.separator import Separator

def extract_vocals(input_dir, vocal_output_dir):
    """
    Extract vocals from audio files using Spleeter.

    Args:
    - input_dir (str): Directory containing full songs.
    - vocal_output_dir (str): Directory to save isolated vocals.

    Returns:
    - None
    """
    os.makedirs(vocal_output_dir, exist_ok=True)
    audio_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]
    separator = Separator('spleeter:2stems')

    for audio_file in audio_files:
        try:
            input_path = os.path.join(input_dir, audio_file)
            output_path = os.path.join(vocal_output_dir, audio_file)
            # Separate vocals
            separator.separate_to_file(input_path, vocal_output_dir)
            print(f"Vocals extracted: {output_path}")
        except Exception as e:
            print(f"Error extracting vocals from {audio_file}: {e}")


def preprocess_audio(vocal_input_dir, mel_output_dir, sample_rate=16000, n_mels=80, frame_length=1024, hop_length=256):
    """
    Preprocess isolated vocal files for CycleGAN-VC2 by converting them to Mel-spectrograms.

    Args:
    - vocal_input_dir (str): Directory containing isolated vocals.
    - mel_output_dir (str): Directory to save Mel-spectrograms.
    - sample_rate (int): Target sample rate.
    - n_mels (int): Number of Mel-frequency bands.
    - frame_length (int): FFT window size.
    - hop_length (int): Hop size for FFT.

    Returns:
    - None
    """
    os.makedirs(mel_output_dir, exist_ok=True)
    vocal_files = [f for f in os.listdir(vocal_input_dir) if f.endswith('.wav')]

    for vocal_file in vocal_files:
        try:
            input_path = os.path.join(vocal_input_dir, vocal_file)
            # Load the audio
            audio, _ = librosa.load(input_path, sr=sample_rate, mono=True)
            # Normalize the audio
            audio = audio / np.max(np.abs(audio))
            # Extract Mel-spectrogram
            mel_spec = librosa.feature.melspectrogram(
                y=audio,
                sr=sample_rate,
                n_fft=frame_length,
                hop_length=hop_length,
                n_mels=n_mels,
                power=2.0
            )
            # Convert to log scale (dB)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
            # Save Mel-spectrogram
            output_path = os.path.join(mel_output_dir, f"{os.path.splitext(vocal_file)[0]}.npy")
            np.save(output_path, mel_spec_db)
            print(f"Mel-spectrogram saved: {output_path}")
        except Exception as e:
            print(f"Error processing {vocal_file}: {e}")


def main():
    # Paths
    input_dir = '/kaggle/input/gan-audio/kishore_songs/kishore_songs'  # Replace with the directory containing full songs
    vocal_output_dir = '/kaggle/working/SongData/kishore_songs/vocals/'  # Directory to save isolated vocals
    mel_output_dir = '/kaggle/working/SongData/kishore_songs/spec'  # Directory to save Mel-spectrograms

    # Step 1: Extract vocals
    print("Extracting vocals...")
    extract_vocals(input_dir, vocal_output_dir)

    # Step 2: Preprocess vocals to generate Mel-spectrograms
    #print("Generating Mel-spectrograms...")
    #preprocess_audio(vocal_output_dir, mel_output_dir)

    input_dir = '/kaggle/input/gan-audio/arijit_songs/arijit_songs'  # Replace with the directory containing full songs
    vocal_output_dir = '/kaggle/working/SongData/arijit_songs/vocals/'  # Directory to save isolated vocals
    mel_output_dir = '/kaggle/working/SongData/arijit_songs/spec'  # Directory to save Mel-spectrograms

    # Step 1: Extract vocals
    print("Extracting vocals...")
    extract_vocals(input_dir, vocal_output_dir)

    # Step 2: Preprocess vocals to generate Mel-spectrograms
    #print("Generating Mel-spectrograms...")
    #preprocess_audio(vocal_output_dir, mel_output_dir)

if __name__ == "__main__":
    main()


In [ ]:
import os
import librosa
import numpy as np

def preprocess_audio_recursive(root_dir, mel_output_dir, sample_rate=16000, n_mels=80, frame_length=1024, hop_length=256):
    """
    Preprocess all `.wav` files in subfolders by converting them to Mel-spectrograms.

    Args:
    - root_dir (str): Root directory containing subfolders with `.wav` files.
    - mel_output_dir (str): Directory to save Mel-spectrogram `.npy` files.
    - sample_rate (int): Target sample rate for the audio files.
    - n_mels (int): Number of Mel-frequency bands.
    - frame_length (int): FFT window size.
    - hop_length (int): Hop size for FFT.

    Returns:
    - None
    """
    os.makedirs(mel_output_dir, exist_ok=True)

    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.wav'):
                try:
                    input_path = os.path.join(subdir, file)
                    
                    # Load the isolated vocal audio
                    audio, _ = librosa.load(input_path, sr=sample_rate, mono=True)
                    
                    # Normalize the audio
                    audio = audio / np.max(np.abs(audio))
                    
                    # Extract Mel-spectrogram
                    mel_spec = librosa.feature.melspectrogram(
                        y=audio,
                        sr=sample_rate,
                        n_fft=frame_length,
                        hop_length=hop_length,
                        n_mels=n_mels,
                        power=2.0
                    )
                    
                    # Convert to log scale (dB)
                    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
                    
                    # Create subdirectory structure in the output directory
                    relative_path = os.path.relpath(subdir, root_dir)
                    mel_subdir = os.path.join(mel_output_dir, relative_path)
                    os.makedirs(mel_subdir, exist_ok=True)
                    
                    # Save the Mel-spectrogram as a `.npy` file
                    output_path = os.path.join(mel_subdir, f"{os.path.splitext(file)[0]}.npy")
                    np.save(output_path, mel_spec_db)
                    print(f"Processed and saved Mel-spectrogram: {output_path}")

                except Exception as e:
                    print(f"Error processing {file} in {subdir}: {e}")

# Example usage
vocal_input_directory = "/kaggle/working/SongData/arijit_songs/vocals"  # Replace with your extracted vocals directory
mel_output_directory = "/kaggle/working/SongData/arijit_songs/spec"   # Directory to save Mel-spectrograms
preprocess_audio_recursive(vocal_input_directory, mel_output_directory)

vocal_input_directory = "/kaggle/working/SongData/kishore_songs/vocals"  # Replace with your extracted vocals directory
mel_output_directory = "/kaggle/working/SongData/kishore_songs/spec"   # Directory to save Mel-spectrograms
preprocess_audio_recursive(vocal_input_directory, mel_output_directory)


In [ ]:
import os
import shutil

def reorganize_vocal_files(root_dir, output_dir):
    """
    Reorganize vocals.npy files into a new folder structure.

    Args:
    - root_dir (str): Path to the root directory containing the 'spec' folder.
    - output_dir (str): Path to the output directory where new 'spec' structure will be created.

    Returns:
    - None
    """
    arijit_source_dir = os.path.join(root_dir, "arijit_songs", "spec")
    kishore_source_dir = os.path.join(root_dir, "kishore_songs", "spec")
    
    # Define output directories
    arijit_output_dir = os.path.join(output_dir, "spec", "arijit")
    kishore_output_dir = os.path.join(output_dir, "spec", "kishore")
    
    # Create output directories
    os.makedirs(arijit_output_dir, exist_ok=True)
    os.makedirs(kishore_output_dir, exist_ok=True)
    
    # Function to process a source directory
    def process_source_dir(source_dir, output_subdir):
        if not os.path.exists(source_dir):
            print(f"Source directory {source_dir} does not exist. Skipping.")
            return
        
        for song_folder in os.listdir(source_dir):
            song_path = os.path.join(source_dir, song_folder)
            if os.path.isdir(song_path):  # Ensure it's a folder
                vocals_file = os.path.join(song_path, "vocals.npy")
                if os.path.exists(vocals_file):
                    # Rename and move vocals.npy
                    new_filename = f"{song_folder}-vocals.npy"
                    output_path = os.path.join(output_subdir, new_filename)
                    shutil.copy(vocals_file, output_path)
                    print(f"Copied: {vocals_file} -> {output_path}")
                else:
                    print(f"vocals.npy not found in {song_path}. Skipping.")
    
    # Process Arijit and Kishore source directories
    process_source_dir(arijit_source_dir, arijit_output_dir)
    process_source_dir(kishore_source_dir, kishore_output_dir)
    print("Reorganization complete.")

# Example usage
root_directory = "/kaggle/working/SongData"  # Replace with the path to your 'SongData' folder
output_directory = "/kaggle/working/specs"  # Replace with the path to your desired output location
reorganize_vocal_files(root_directory, output_directory)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm

# Generator
class Generator(nn.Module):
    def __init__(self, input_dim=80, hidden_dim=256, num_residual_blocks=6):
        super(Generator, self).__init__()
        layers = [
            nn.Conv1d(input_dim, hidden_dim, kernel_size=15, padding=7),
            nn.InstanceNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        ]

        # Downsampling
        for _ in range(2):
            layers.append(
                nn.Conv1d(hidden_dim, hidden_dim * 2, kernel_size=5, stride=2, padding=2)
            )
            layers.append(nn.InstanceNorm1d(hidden_dim * 2))
            layers.append(nn.ReLU(inplace=True))
            hidden_dim *= 2

        # Residual blocks
        for _ in range(num_residual_blocks):
            layers.append(ResidualBlock(hidden_dim))

        # Upsampling
        for _ in range(2):
            layers.append(
                nn.ConvTranspose1d(hidden_dim, hidden_dim // 2, kernel_size=5, stride=2, padding=2, output_padding=1)
            )
            layers.append(nn.InstanceNorm1d(hidden_dim // 2))
            layers.append(nn.ReLU(inplace=True))
            hidden_dim //= 2

        layers.append(nn.Conv1d(hidden_dim, input_dim, kernel_size=15, padding=7))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv1d(dim, dim, kernel_size=3, padding=1),
            nn.InstanceNorm1d(dim),
            nn.ReLU(inplace=True),
            nn.Conv1d(dim, dim, kernel_size=3, padding=1),
            nn.InstanceNorm1d(dim)
        )

    def forward(self, x):
        return x + self.block(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim=80):
        super(Discriminator, self).__init__()
        layers = [
            nn.Conv1d(input_dim, 128, kernel_size=15, stride=1, padding=7),
            nn.LeakyReLU(0.2, inplace=True),
        ]

        num_filters = 128
        for _ in range(3):
            layers.append(
                nn.Conv1d(num_filters, num_filters * 2, kernel_size=15, stride=2, padding=7)
            )
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            num_filters *= 2

        layers.append(nn.Conv1d(num_filters, 1, kernel_size=3, stride=1, padding=1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Dataset
class SpectrogramDataset(Dataset):
    def __init__(self, root_dir, min_length=None, max_length=None):
        self.files = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith('.npy')]
        
        # Load and filter spectrograms based on length
        self.spectrograms = []
        for file in self.files:
            try:
                spec = np.load(file)
                
                # Print shape for debugging
                print(f"Loaded {file} with shape: {spec.shape}")
                
                # Handle various possible shapes
                if spec.ndim == 4:
                    spec = spec.squeeze()
                
                # Ensure shape is (80, time_steps)
                if spec.ndim == 3:
                    if spec.shape[0] == 1:
                        spec = spec.squeeze(0)
                    elif spec.shape[2] == 80:
                        spec = spec.transpose(0, 2).squeeze(0)
                elif spec.ndim == 2:
                    if spec.shape[1] == 80:
                        spec = spec.T
                
                # Verify shape
                assert spec.shape[0] == 80, f"Unexpected spectrogram shape for {file}: {spec.shape}"
                
                # Apply length filtering if specified
                if min_length is not None and spec.shape[1] < min_length:
                    continue
                if max_length is not None and spec.shape[1] > max_length:
                    continue
                
                self.spectrograms.append(spec)
            except Exception as e:
                print(f"Error processing file {file}: {e}")

    def __len__(self):
        return len(self.spectrograms)

    def __getitem__(self, idx):
        return self.spectrograms[idx]  # Return NumPy array directly

def pad_to_max_size(tensor1, tensor2):
    # Get the maximum size along the time dimension
    max_size = max(tensor1.shape[2], tensor2.shape[2])
    
    # Pad both tensors to the max size
    tensor1 = torch.nn.functional.pad(tensor1, (0, max_size - tensor1.shape[2]))
    tensor2 = torch.nn.functional.pad(tensor2, (0, max_size - tensor2.shape[2]))
    
    return tensor1, tensor2

# Utility function to pad or trim tensors to a consistent length
def pad_or_trim_tensor(tensor, target_length):
    # Handle 4D tensor from some spectrogram formats
    if tensor.ndim == 4:
        # Squeeze out unnecessary dimensions
        tensor = tensor.squeeze(0)  # Removing the extra dimension

    # If tensor is already a PyTorch tensor, convert to NumPy if needed
    if isinstance(tensor, torch.Tensor):
        tensor = tensor.numpy()

    # Ensure tensor is of shape (80, time_steps)
    if tensor.ndim == 3:
        # If shape is (1, 80, time_steps) or (1, time_steps, 80)
        if tensor.shape[0] == 1:
            tensor = tensor.squeeze(0)  # Removing the first dimension (channel dimension)
        elif tensor.shape[2] == 80:
            tensor = tensor.transpose(0, 2).squeeze(0)
    elif tensor.ndim == 2:
        # If shape is (time_steps, 80)
        if tensor.shape[1] == 80:
            tensor = tensor.T

    # Ensure shape is (80, time_steps)
    assert tensor.shape[0] == 80, f"Unexpected tensor shape: {tensor.shape}"

    current_length = tensor.shape[1]

    if current_length == target_length:
        return torch.from_numpy(tensor)  # No need to add batch dimension here

    if current_length < target_length:
        # Pad with zeros
        pad_size = target_length - current_length
        padded = np.pad(tensor, ((0, 0), (0, pad_size)), mode='constant')
        return torch.from_numpy(padded)  # No need to add batch dimension here
    else:
        # Trim 
        return torch.from_numpy(tensor[:, :target_length])  # No need to add batch dimension here



# Loss Functions
mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

# Training
# Training
def train_cycle_gan(train_A_dir, train_B_dir, output_dir, epochs=50, batch_size=1, lr=0.0002):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Determine common spectrogram length
    def get_common_length(dataset):
        lengths = [spec.shape[1] if isinstance(spec, np.ndarray) else spec.shape[1] for spec in dataset.spectrograms]
        return min(lengths)

    # Load datasets
    dataset_A = SpectrogramDataset(train_A_dir)
    dataset_B = SpectrogramDataset(train_B_dir)
    
    print(f"Dataset A: {len(dataset_A)} spectrograms")
    print(f"Dataset B: {len(dataset_B)} spectrograms")
    
    # Debug: print first few spectrogram shapes
    if dataset_A.spectrograms:
        print("First spectrogram A shape:", dataset_A.spectrograms[0].shape)
    if dataset_B.spectrograms:
        print("First spectrogram B shape:", dataset_B.spectrograms[0].shape)
    
    # Find common length
    common_length = min(get_common_length(dataset_A), get_common_length(dataset_B))
    print(f"Common length: {common_length}")
    
    # Create dataloaders with padding/trimming
    dataloader_A = DataLoader(dataset_A, batch_size=batch_size, shuffle=True, 
                               collate_fn=lambda x: torch.stack([pad_or_trim_tensor(spec, common_length) for spec in x]))
    dataloader_B = DataLoader(dataset_B, batch_size=batch_size, shuffle=True, 
                               collate_fn=lambda x: torch.stack([pad_or_trim_tensor(spec, common_length) for spec in x]))

    # Rest of the code remains the same...

    # Initialize models
    G_A2B = Generator()
    G_B2A = Generator()
    D_A = Discriminator()
    D_B = Discriminator()

    # Optimizers
    opt_G = optim.Adam(list(G_A2B.parameters()) + list(G_B2A.parameters()), lr=lr, betas=(0.5, 0.999))
    opt_D_A = optim.Adam(D_A.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_D_B = optim.Adam(D_B.parameters(), lr=lr, betas=(0.5, 0.999))

    # Training loop
    for epoch in range(epochs):
        total_loss_G = 0
        total_loss_D_A = 0
        total_loss_D_B = 0
        
        progress_bar = tqdm(zip(dataloader_A, dataloader_B), total=min(len(dataloader_A), len(dataloader_B)))
        for real_A, real_B in progress_bar:
            # Verify tensor shapes
            assert real_A.shape[1] == 80, f"real_A has incorrect shape: {real_A.shape}"
            assert real_B.shape[1] == 80, f"real_B has incorrect shape: {real_B.shape}"
            assert real_A.shape[2] == real_B.shape[2], f"Tensor lengths do not match: {real_A.shape}, {real_B.shape}"

            # Train Generators
            fake_B = G_A2B(real_A)
            fake_A = G_B2A(real_B)
            cycle_A = G_B2A(fake_B)
            cycle_B = G_A2B(fake_A)

            cycle_A, real_A = pad_to_max_size(cycle_A, real_A)
            cycle_B, real_B = pad_to_max_size(cycle_B, real_B)

            loss_cycle = l1_loss(cycle_A, real_A) + l1_loss(cycle_B, real_B)
            loss_identity = l1_loss(G_A2B(real_B), real_B) + l1_loss(G_B2A(real_A), real_A)
            loss_G_A2B = mse_loss(D_B(fake_B), torch.ones_like(D_B(fake_B)))
            loss_G_B2A = mse_loss(D_A(fake_A), torch.ones_like(D_A(fake_A)))
            loss_G = loss_G_A2B + loss_G_B2A + 10 * loss_cycle + 5 * loss_identity

            opt_G.zero_grad()
            loss_G.backward()
            opt_G.step()

            # Train Discriminators
            loss_D_A = (mse_loss(D_A(real_A), torch.ones_like(D_A(real_A))) +
                        mse_loss(D_A(fake_A.detach()), torch.zeros_like(D_A(fake_A)))) * 0.5
            loss_D_B = (mse_loss(D_B(real_B), torch.ones_like(D_B(real_B))) +
                        mse_loss(D_B(fake_B.detach()), torch.zeros_like(D_B(fake_B)))) * 0.5

            opt_D_A.zero_grad()
            loss_D_A.backward()
            opt_D_A.step()

            opt_D_B.zero_grad()
            loss_D_B.backward()
            opt_D_B.step()

            # Accumulate losses for logging
            total_loss_G += loss_G.item()
            total_loss_D_A += loss_D_A.item()
            total_loss_D_B += loss_D_B.item()

            # Update progress bar
            progress_bar.set_description(f"Epoch {epoch + 1}/{epochs}")
            progress_bar.set_postfix({
                'Loss G': loss_G.item(), 
                'Loss D_A': loss_D_A.item(), 
                'Loss D_B': loss_D_B.item()
            })

        # Print average losses for the epoch
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Avg Loss G: {total_loss_G / len(progress_bar):.4f}")
        print(f"Avg Loss D_A: {total_loss_D_A / len(progress_bar):.4f}")
        print(f"Avg Loss D_B: {total_loss_D_B / len(progress_bar):.4f}")

        # Save models
        torch.save(G_A2B.state_dict(), os.path.join(output_dir, f'G_A2B_epoch_{epoch + 1}.pth'))
        torch.save(G_B2A.state_dict(), os.path.join(output_dir, f'G_B2A_epoch_{epoch + 1}.pth'))
        torch.save(D_A.state_dict(), os.path.join(output_dir, f'D_A_epoch_{epoch + 1}.pth'))
        torch.save(D_B.state_dict(), os.path.join(output_dir, f'D_B_epoch_{epoch + 1}.pth'))

# Paths (modify these to your specific directories)
train_A_dir = "/kaggle/working/specs/spec/arijit"
train_B_dir = "/kaggle/working/specs/spec/kishore"
output_dir = "/kaggle/working/saved_models"

# Ensure to replace the paths with your actual directories
train_cycle_gan(train_A_dir, train_B_dir, output_dir)

Loaded /kaggle/working/specs/spec/arijit/song34-vocals.npy with shape: (80, 14554)
Loaded /kaggle/working/specs/spec/arijit/song9-vocals.npy with shape: (80, 13792)
Loaded /kaggle/working/specs/spec/arijit/song42-vocals.npy with shape: (80, 18577)
Loaded /kaggle/working/specs/spec/arijit/song6-vocals.npy with shape: (80, 15008)
Loaded /kaggle/working/specs/spec/arijit/song37-vocals.npy with shape: (80, 17566)
Loaded /kaggle/working/specs/spec/arijit/song14-vocals.npy with shape: (80, 16190)
Loaded /kaggle/working/specs/spec/arijit/song45-vocals.npy with shape: (80, 18143)
Loaded /kaggle/working/specs/spec/arijit/song40-vocals.npy with shape: (80, 18141)
Loaded /kaggle/working/specs/spec/arijit/song32-vocals.npy with shape: (80, 16598)
Loaded /kaggle/working/specs/spec/arijit/song20-vocals.npy with shape: (80, 19540)
Loaded /kaggle/working/specs/spec/arijit/song41-vocals.npy with shape: (80, 18255)
Loaded /kaggle/working/specs/spec/arijit/song21-vocals.npy with shape: (80, 18324)
Loaded

Epoch 1/200: 100%|██████████| 45/45 [12:57<00:00, 17.29s/it, Loss G=1.26e+3, Loss D_A=0.0527, Loss D_B=0.027]  


Epoch 1/200
Avg Loss G: 1513.5315
Avg Loss D_A: 56.3634
Avg Loss D_B: 36.9232


Epoch 2/200: 100%|██████████| 45/45 [12:40<00:00, 16.91s/it, Loss G=692, Loss D_A=0.0811, Loss D_B=0.0515]    


Epoch 2/200
Avg Loss G: 1036.4327
Avg Loss D_A: 0.0698
Avg Loss D_B: 0.0502


...

Epoch 5/200: 100%|██████████| 45/45 [12:23<00:00, 16.53s/it, Loss G=326, Loss D_A=0.105, Loss D_B=0.0877] 


Epoch 5/200
Avg Loss G: 329.1983
Avg Loss D_A: 0.1015
Avg Loss D_B: 0.0885


Epoch 6/200: 100%|██████████| 45/45 [12:16<00:00, 16.36s/it, Loss G=311, Loss D_A=0.0584, Loss D_B=0.0824]


Epoch 6/200
Avg Loss G: 306.1470
Avg Loss D_A: 0.0917
Avg Loss D_B: 0.0837


Epoch 7/200:  38%|███▊      | 17/45 [04:35<07:36, 16.32s/it, Loss G=267, Loss D_A=0.112, Loss D_B=0.243]  